# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.36.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-176650
aml-quickstarts-176650
southcentralus
9a7511b8-150f-4a58-8528-3e7d50216c31


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'ml-experiment-1'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
ml-experiment-1,quick-starts-ws-176650,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [5]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 1)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded............
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [7]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "divorce-ds"
description_text = ""

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key]
        print('dataset found in workspace')

if not found:
        print('dataset NOT found in workspace')
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/joangerard/project-azure-piepline/master/divorce_data.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data, separator=';')        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

dataset found in workspace


,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q46,Q47,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Divorce
count,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,...,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000
mean,1.776471,1.652941,1.764706,1.482353,1.541176,0.747059,0.494118,1.452941,1.458824,1.576471,...,2.552941,2.270588,2.741176,2.382353,2.429412,2.476471,2.517647,2.241176,2.011765,0.494118
std,1.627257,1.468654,1.415444,1.504327,1.632169,0.904046,0.898698,1.546371,1.557976,1.421529,...,1.371786,1.586841,1.137348,1.511587,1.405090,1.260238,1.476537,1.505634,1.667611,0.501442
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,0.000000
50%,2.000000,2.000000,2.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,2.000000,...,3.000000,2.000000,3.000000,3.000000,2.000000,3.000000,3.000000,2.000000,2.000000,0.000000
75%,3.000000,3.000000,3.000000,3.000000,3.000000,1.000000,1.000000,3.000000,3.000000,3.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,1.000000


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [8]:
dataset.take(5).to_pandas_dataframe()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q46,Q47,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Divorce
0,2,2,4,1,0,0,0,0,0,0,...,2,1,3,3,3,2,3,2,1,1
1,4,4,4,4,4,0,0,4,4,4,...,2,2,3,4,4,4,4,2,2,1
2,2,2,2,2,1,3,2,1,1,2,...,3,2,3,1,1,1,2,2,2,1
3,3,2,3,2,3,3,3,3,3,3,...,2,2,3,3,3,3,2,2,2,1
4,2,2,1,1,1,1,0,0,0,0,...,2,1,2,3,2,2,2,1,0,1


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [20]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Divorce",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [21]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [22]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [23]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [24]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [65ca1882][26f9382b-b1b6-4b18-9ac2-78c692c74b4a], (This step will run and generate new outputs)
Submitted PipelineRun 0eb4ce59-aa7a-458a-81e4-be950113565b
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/0eb4ce59-aa7a-458a-81e4-be950113565b?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-176650/workspaces/quick-starts-ws-176650&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


In [25]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [26]:
pipeline_run.wait_for_completion()

PipelineRunId: 0eb4ce59-aa7a-458a-81e4-be950113565b
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/0eb4ce59-aa7a-458a-81e4-be950113565b?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-176650/workspaces/quick-starts-ws-176650&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: Running


StepRunId: 087c0565-8aa8-4e36-bb34-3be30f0e5fd3
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/087c0565-8aa8-4e36-bb34-3be30f0e5fd3?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-176650/workspaces/quick-starts-ws-176650&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/py

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [27]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/087c0565-8aa8-4e36-bb34-3be30f0e5fd3/metrics_data, 1 files out of an estimated total of 1


In [28]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_22,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_4,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_26,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_33,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_29,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_34,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_28,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_31,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_30,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_0,...,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_37,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_14,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_19,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_21,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_23,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_18,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_25,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_32,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_39,087c0565-8aa8-4e36-bb34-3be30f0e5fd3_38
weighted_accuracy,[0.9540787594368496],[0.9720097939196082],[0.9775270352989185],[0.9720097939196082],[0.9775270352989185],[0.9775270352989185],[0.9720097939196082],[0.9775270352989185],[0.36554046633408604],[0.9775270352989185],...,[0.9775270352989185],[0.9485615180575392],[0.9651132421954702],[0.965802897367884],[0.9775270352989185],[0.9720097939196082],[0.9533891042644358],[0.9837339318506426],[0.9782166904713323],[0.9775270352989185]
accuracy,[0.9529411764705884],[0.9705882352941178],[0.9764705882352942],[0.9705882352941178],[0.9764705882352942],[0.9764705882352942],[0.9705882352941178],[0.9764705882352942],[0.4235294117647059],[0.9764705882352942],...,[0.9764705882352942],[0.9470588235294117],[0.9647058823529413],[0.9647058823529413],[0.9764705882352942],[0.9705882352941178],[0.9529411764705884],[0.9823529411764707],[0.9764705882352942],[0.9764705882352942]
precision_score_micro,[0.9529411764705884],[0.9705882352941178],[0.9764705882352942],[0.9705882352941178],[0.9764705882352942],[0.9764705882352942],[0.9705882352941178],[0.9764705882352942],[0.4235294117647059],[0.9764705882352942],...,[0.9764705882352942],[0.9470588235294117],[0.9647058823529413],[0.9647058823529413],[0.9764705882352942],[0.9705882352941178],[0.9529411764705884],[0.9823529411764707],[0.9764705882352942],[0.9764705882352942]
norm_macro_recall,[0.9022222222222223],[0.9369444444444444],[0.9494444444444443],[0.9369444444444444],[0.9494444444444443],[0.9494444444444443],[0.9369444444444444],[0.9494444444444443],[0.0],[0.9494444444444443],...,[0.9494444444444443],[0.8897222222222222],[0.9272222222222222],[0.9258333333333333],[0.9494444444444443],[0.9369444444444444],[0.9036111111111111],[0.9605555555555554],[0.9480555555555554],[0.9494444444444443]
precision_score_macro,[0.9573785936285937],[0.9704594017094017],[0.9767094017094017],[0.9704594017094017],[0.9767094017094017],[0.9767094017094017],[0.9704594017094017],[0.9767094017094017],[0.21176470588235294],[0.9767094017094017],...,[0.9767094017094017],[0.9488721001221002],[0.9686285936285935],[0.9660149572649572],[0.9767094017094017],[0.9704594017094017],[0.9568230380730383],[0.9822649572649572],[0.9772649572649572],[0.9767094017094017]
f1_score_micro,[0.9529411764705884],[0.9705882352941178],[0.9764705882352942],[0.9705882352941178],[0.9764705882352942],[0.9764705882352942],[0.9705882352941178],[0.9764705882352942],[0.4235294117647059],[0.9764705882352942],...,[0.9764705882352942],[0.9470588235294117],[0.9647058823529413],[0.9647058823529413],[0.9764705882352942],[0.9705882352941178],[0.9529411764705884],[0.9823529411764707],[0.9764705882352942],[0.9764705882352942]
average_precision_score_macro,[0.9952553310886645],[0.99875],[0.9967628205128205],[0.9975000000000002],[0.99875],[1.0],[0.9975000000000002],[1.0],[0.5],[0.9967628205128205],...,[1.0],[0.9951781705948374],[1.0],[0.99875],[1.0],[0.99875],[0.9951781705948374],[1.0],[1.0],[1.0]
log_loss,[0.29889844694038553],[0.14099732417688277],[0.07835484484355444],[0.16687098498146732],[0.17428125960766222],[0.05235003706819462],[0.13775454109486338],[0.05768108765235188],[0.7013061098925675],[0.09169236604145156],...,[0.07056990675312638],[0.32475337134579546],[0.150289602

### Retrieve the Best Model

In [16]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/ce995052-44fd-4711-a9ce-95ec83ca5407/model_data, 1 files out of an estimated total of 1


In [17]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='classification', working_dir='/m...
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=None, reg_alpha=0.3684210526315789, reg_lambda=1, subsample=1))], verbose=False))], flatten_transform=None, weights=[0.14285714285714285, 0.21428571428571427, 0.07142857142857142, 0.2857142857142857, 0.2857142857142857]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [18]:
best_model.steps

[('datatransformer',
  DataTransformer(
      task='classification',
      is_onnx_compatible=False,
      enable_feature_sweeping=True,
      enable_dnn=False,
      force_text_dnn=False,
      feature_sweeping_timeout=86400,
      featurization_config=None,
      is_cross_validation=False,
      feature_sweeping_config={}
  )),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(
      estimators=[('0', Pipeline(
          memory=None,
          steps=[('maxabsscaler', MaxAbsScaler(
              copy=True
          )), ('lightgbmclassifier', LightGBMClassifier(
              min_data_in_leaf=20,
              random_state=None,
              n_jobs=1,
              problem_info=ProblemInfo(
                  gpu_training_param_dict={'processing_unit_type': 'cpu'}
              )
          ))],
          verbose=False
      )), ('34', Pipeline(
          memory=None,
          steps=[('sparsenormalizer', Normalizer(
              copy=True,
              norm='max'
   

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [19]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [20]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

In [21]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

,0,1
0,28946,312
1,1408,2284


## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [22]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bankmarketing Train,b07bf35a-a28c-4f85-8347-a2428cb1aeaa,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [24]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [25]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [26]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  f0a05ee7-562b-4835-ae99-49654c78b4a5


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [27]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …